In [4]:
import json
from anonymizer.anon_helper import *
import pandas as pd
import numpy as np
import random

In [9]:
!ls -a

.                  .git               exam_structure.txt readme.md
..                 .ipynb_checkpoints grade_exams.py     reporter.py
.DS_Store          Untitled.ipynb     gradebook          student_ids.txt
.Rhistory          anonymizer         process_midterm


In [160]:
conv_df = pd.DataFrame(columns=["name", "perm_id", "temp_id", "rm", "email", "slack"])


with open('student_ids.txt') as students:
    temp  = json.load(students)
    a = 1
    for p in temp:
        print(p)
        conv_df = conv_df.append(p, ignore_index = True)

print(a)
temp2 = json.loads(conv_df.to_json(orient='records'))
print(temp2)

{'name': 'Sarah Lancaster', 'perm_id': '0001', 'temp_id': '2013', 'rm': '1', 'email': 'sarah.lancaster@gmail.com', 'slack': 'itssarah'}
{'name': 'Arthur Morgan', 'perm_id': '0002', 'temp_id': '1293', 'rm': '2', 'email': 'arthur.morgan@gmail.com', 'slack': 'AMorgan'}
{'name': 'Lenny Truville', 'perm_id': '0003', 'temp_id': '1238', 'rm': '1', 'email': 'ltruville002@gmail.com', 'slack': ''}
{'name': 'Jackie Trudeau', 'perm_id': '0004', 'temp_id': '2039', 'rm': '3', 'email': 'jackie.trudeau@gmail.com', 'slack': 'JackieTru'}
1
[{'name': 'Sarah Lancaster', 'perm_id': '0001', 'temp_id': '2013', 'rm': '1', 'email': 'sarah.lancaster@gmail.com', 'slack': 'itssarah'}, {'name': 'Arthur Morgan', 'perm_id': '0002', 'temp_id': '1293', 'rm': '2', 'email': 'arthur.morgan@gmail.com', 'slack': 'AMorgan'}, {'name': 'Lenny Truville', 'perm_id': '0003', 'temp_id': '1238', 'rm': '1', 'email': 'ltruville002@gmail.com', 'slack': ''}, {'name': 'Jackie Trudeau', 'perm_id': '0004', 'temp_id': '2039', 'rm': '3', '

In [161]:
def json_format_preserver(df, json_filename, filepath = "./"):
    '''
    A shortcut way to make sure that the JSON file has its format preserved.
    Takes:
    df: a dataframe that wants to be converted to JSON format.
    json_filename: the filename of the JSON file.
    
    Returns:
    None
    '''
    
    formatter = json.loads(df.to_json(orient='records'))
    
    with open(filepath+json_filename, 'w') as f:
        json.dump(formatter, f, indent=4)

In [163]:
json_format_preserver(df = conv_df, json_filename = 'abc.txt')

In [69]:
# INITIAL CONDITIONS
filepath = "./test_exams/"
filetype = ".R"
exam_name = "R_EDA"
output_filename = "conv_test.csv"
decrypt = False
student_filepath = "student_ids.txt"
########

# start create_key_df

exam_desc = "_"+exam_name+filetype

exams = [x for x in glob.glob(filepath+"*"+filetype) if ("solution" not in x.lower()) and \
                                               ("anonomizer" not in x.lower())]
student_names = [label_processor(x) for x in exams]

file_df = pd.DataFrame({"orig_file": exams, "name": student_names})

# should be refactored into "load_json_to_df"
linking_df = load_json_to_df(student_filepath)
        
left_overs = [student for student in file_df['name'].values if student not in linking_df['name'].values]


masked_df = linking_df[linking_df['name'].isin(student_names)]
masked_df['temp_id'] = random.sample(range(1000,9999), masked_df.shape[0])

# make sure that users that did not submit an exam have a unique temp_id code.
unsubmitted = linking_df[~linking_df['name'].isin(student_names)]
unsubmitted['temp_id'] = '0000'

# clean up a little bit
linking_df = pd.concat([masked_df, unsubmitted], axis = 0)

linking_df.sort_values('perm_id', inplace = True)

json_format_preserver(linking_df, json_filename = student_filepath)

# don't need everything moving on, just the name
reduced_df = linking_df[['name', 'temp_id']]

# only want to encrypt exams that have names we know!! (a better way to do this would be
# if we could possible get the email tagged onto the file when we download this; so the user
# account would be the thing we link on, not if they spelled their name correctly (reduce
# user error)).
file_df = pd.merge(reduced_df, file_df, how = 'inner', on='name')

file_df['encrypt_file'] = [filepath+str(ID)+exam_desc for ID in file_df['temp_id'].values]

file_df.to_csv(filepath+output_filename)

return file_df[['orig_file', 'encrypt_file']]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
filepath = "./R Exams/"
filetype = ".R"
exam_name = "R_EDA"
output_filename = "conv_test.csv"
decrypt = False
student_filepath = "student_ids.txt"
########

# start create_key_df

exam_desc = "_"+exam_name+filetype

exams = [x for x in glob.glob(filepath+"*"+filetype) if ("solution" not in x.lower()) and \
                                               ("anonomizer" not in x.lower())]
student_names = [label_processor(x) for x in exams]

id_numbers = ["000" + str(idx) if idx < 10 else "00"+str(idx) for idx in range(len(student_names))]

In [12]:
json_list = [{'name':name, 'perm_id':idx} for name, idx in zip(student_names, id_numbers)]

with open('stu_info.txt', 'w') as f:
    json.dump(json_list, f, indent=4)

In [4]:
re.findall("[A-Z][a-z]+", "TestExample")

['Test', 'Example']

In [155]:
distributor('TA_data.txt', file_df)

Creating zip file for TA Sam's exam(s).
Writing exam:  ./test_exams/8732_R_EDA.R
TA  Charlie does not have any exams to grade.
Creating zip file for TA Wenjun's exam(s).
Writing exam:  ./test_exams/6482_R_EDA.R
Creating zip file for TA Simon's exam(s).
Writing exam:  ./test_exams/2778_R_EDA.R


,email,exams_to_grade
0,sam@nycdsa.com,./test_exams/8732_R_EDA.R
1,charlie@nycdsa.com,None
2,wenjun@nycdsa.com,./test_exams/6482_R_EDA.R
3,simon@nycdsa.com,./test_exams/2778_R_EDA.R


In [94]:
names = ['Sam', 'Charlie', 'Wenjun', 'Simon']
emails = ['sam@nycdsa.com', 'charlie@nycdsa.com', 'wenjun@nycdsa.com', 'simon@nycdsa.com']

TA_json = []
for name, email in zip(names, emails):
    TA_json.append({'name': name, 'email': email})


    
with open('TA_data.txt', 'w') as f:
    json.dump(TA_json, f, indent=4)

In [107]:
exams_to_send = file_df['encrypt_file'].values
np.random.shuffle(exams_to_send)
exams_to_send

array(['./test_exams/6482_R_EDA.R', './test_exams/8732_R_EDA.R',
       './test_exams/2778_R_EDA.R'], dtype=object)

In [145]:
import zipfile as zp